## 11.3 GTFS Schedule Analysis for the City of Genova

In [ ]:
pip install gtfs_functions

In [ ]:
pip install psycopg2-binary

In [ ]:
pip install geoalchemy2

In [ ]:
pip install sqlalchemy 

In [ ]:
pip install dash

In [ ]:
pip install dash_bootstrap_components

In [1]:
# Libraries to be used in the following cells

import numpy as np
import pandas as pd
import geopandas as gpd
import gtfs_functions as gtfs

# Database interaction
import psycopg2
import psycopg2.extras as extras
from psycopg2 import Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium as fl
from IPython.display import display, HTML, clear_output


# Configure plot style
plt.style.use("fivethirtyeight")
plt.rcParams.update({'font.size': 9})

In [2]:
# Connection parameters
# Required to be run before the following cells
db_host = "localhost"
db_port = "5432"
db_user = "postgres"
db_pass = "postgres"
db_name = "genova"

def connect_postgres(db_host, db_port, db_user, db_pass, db_name):
    """
    Connects to a PostgreSQL database using the given credentials.
    
    Parameters:
    - db_host: Database host
    - db_port: Database port
    - db_user: Username
    - db_pass: Password
    - db_name: Database name
    
    Returns:
    - cur: Cursor object for executing SQL queries
    """
    try:
        # Establish connection
        connection = psycopg2.connect(
            host=db_host,
            port=db_port,
            user=db_user,
            password=db_pass,
            database=db_name
        )
        # Set auto-commit for the session
        connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        
        # Create a cursor for executing queries
        cur = connection.cursor()
        
        # Fetch and print PostgreSQL details
        print("Connected to PostgreSQL:", connection.get_dsn_parameters())
        cur.execute("SELECT version();")
        print("PostgreSQL version:", cur.fetchone(), "\n")
        
        return cur

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL:", error)

In [ ]:
# Required before visualization scripts

def fetch_table_sample(cur, table_name, limit=5):
    """
    Fetches a sample of data from the specified table.
    
    Parameters:
    - cur: PostgreSQL cursor object
    - table_name: Name of the table to query
    - limit: Number of records to fetch (default=5)
    
    Returns:
    - DataFrame with fetched records
    """
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    cur.execute(query)
    columns = [desc[0] for desc in cur.description]
    return pd.DataFrame(cur.fetchall(), columns=columns)

# Connect to PostgreSQL
cur = connect_postgres(db_host, db_port, db_user, db_pass, db_name)

# Fetch data samples from different tables
calendar_sample_df = fetch_table_sample(cur, 'calendar')
calendar_dates_sample_df = fetch_table_sample(cur, 'calendar_dates')
routes_sample_df = fetch_table_sample(cur, 'routes')
shapes_sample_df = fetch_table_sample(cur, 'shapes')
stop_times_sample_df = fetch_table_sample(cur, 'stop_times')
stops_sample_df = fetch_table_sample(cur, 'stops')
trips_sample_df = fetch_table_sample(cur, 'trips')
arrivals_departures_sample_df = fetch_table_sample(cur, 'arrivals_departures')
connections_sample_df = fetch_table_sample(cur, 'connections')

In [ ]:
def visualize_network(cur):
    """
    Visualizes the transit network using the 'shapes_aggregated' view from PostgreSQL.
    
    Parameters:
    - cur: PostgreSQL cursor object defined above
    """
    # Query GeoDataFrame
    sql = 'SELECT * FROM shapes_aggregated;'
    shapes_gdf = gpd.GeoDataFrame.from_postgis(sql, cur.connection, geom_col='shape', crs='EPSG:4326')
    
    # Display basic plot
    fig, ax = plt.subplots(1, 1, figsize=(17.5, 0.01))
    ax.set_title('Genova Network Map (all shapes)', size=15)
    ax.axis("off")
    plt.show()

    # Create interactive Folium map
    map_all = shapes_gdf.explore(
        name="All Routes",
        marker_type='circle',
        marker_kwds={'radius': 3, 'fill': True}
    )
    fl.LayerControl().add_to(map_all)
    return map_all

# Visualize network
network_map = visualize_network(cur)
network_map 

In [ ]:
# Section 11.2.2. - Visualizing Transport Routes

import folium as fl
import geopandas as gpd

def map_individual_lines(cur):
    """
    Creates an interactive map to visualize individual transit lines from the 'shapes_aggregated' view.
    
    Parameters:
    - cur: PostgreSQL cursor object to execute SQL queries
    """
    # SQL query to get the transit line shapes  
    sql = 'SELECT * FROM shapes_aggregated;'
    shapes_gdf = gpd.GeoDataFrame.from_postgis(sql, cur.connection, geom_col='shape', crs='EPSG:4326')

    # Initialize the map centered on Riga
    map_indiv_lines = fl.Map(location=[44.4264, 8.91519], tiles='CartoDB positron', zoom_start=13, control_scale=True)

    # Add each route as a FeatureGroup
    for shape_id, shape_group in shapes_gdf.groupby('shape_id'):
        # Initialize a feature group for each individual line (shape_id)
        feature_group = fl.FeatureGroup(name=f"Route {shape_id}", show=True)  # Initially show all routes by default
        
        # Add each LineString geometry as a PolyLine
        for geometry in shape_group.geometry:
            if geometry.geom_type == 'LineString':
                coords = [(lat, lon) for lon, lat in geometry.coords]  # Reverse coords from (lon, lat) to (lat, lon)
                fl.PolyLine(locations=coords, color="blue", weight=2, opacity=0.8).add_to(feature_group)

        # Add the feature group to the map
        feature_group.add_to(map_indiv_lines)

    # Add a layer control to toggle between different routes
    fl.LayerControl(collapsed=False).add_to(map_indiv_lines)

    return map_indiv_lines

individual_lines_map = map_individual_lines(cur)
display(individual_lines_map)

In [ ]:
#Script for Section 11.2.3 

def load_gtfs_feed(file_path, start_date, end_date):
    """
    Loads the GTFS data from a ZIP file and plots the segments.
    
    Parameters:
    - file_path: Path to the GTFS zip file
    - start_date: Start date of the feed (format: YYYY-MM-DD)
    - end_date: End date of the feed (format: YYYY-MM-DD)
    """
    # Load GTFS feed
    feed = gtfs.Feed(file_path, start_date=start_date, end_date=end_date)
    segments_gdf = feed.segments

    # Plot GTFS segments
    segments_gdf.plot()
    return segments_gdf

# Load and plot GTFS feed
gtfs_file = "/Users/avaisman/tmp/GTFS_AMT_GENOVA.zip"
gtfs_segments = load_gtfs_feed(gtfs_file, '2024-01-16', '2024-03-31')

In [ ]:
from sqlalchemy import create_engine

def save_to_postgis(gdf, table_name, db_host, db_port, db_user, db_pass, db_name):
    """
    Saves a GeoDataFrame to a PostgreSQL/PostGIS database.
    
    Parameters:
    - gdf: GeoDataFrame to save
    - table_name: Name of the table where data will be stored
    - db_host: Database host
    - db_port: Database port (default is 5432 for PostgreSQL)
    - db_user: Database user
    - db_pass: Database password
    - db_name: Name of the database
    """
    # Construct the connection URL using the provided parameters
    db_url = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
    
    # Create the SQLAlchemy engine using the constructed URL
    engine = create_engine(db_url)
    
    # Save the GeoDataFrame to PostGIS (PostgreSQL) using to_postgis()
    gdf.to_postgis(table_name, engine, if_exists='replace')
    
    print(f"Data saved to {table_name} table in the {db_name} database.")

# Example usage
save_to_postgis(gtfs_segments, "segments", db_host, db_port, db_user, db_pass, db_name)

In [15]:
#Script for Section 11.2.4 (prepare for visualization)
import psycopg2
import folium as fl
from folium.features import GeoJsonTooltip
import geopandas as gpd

# Function to load segment data directly from PostGIS into a GeoDataFrame
def load_segments_from_postgis(conn):
    """
    Loads the segment data from a PostGIS database into a GeoDataFrame.
    
    Parameters:
    - conn: PostgreSQL connection object
    
    Returns:
    - GeoDataFrame containing segment information and geometries
    """
    query = """
    SELECT c.from_stop_id, c.from_stop_name, c.to_stop_id, c.to_stop_name, s.geometry, 
        count(trip_id) AS count, string_agg(distinct c.route_short_name, ',') AS routes
    FROM connections AS c 
    JOIN segments AS s 
    ON c.route_id=s.route_id AND c.direction_id=s.direction_id AND c.from_stop_id=s.start_stop_id 
       AND c.to_stop_id=s.end_stop_id
    WHERE date between '2024-01-16' and '2024-03-31'
    GROUP BY c.from_stop_id, c.from_stop_name, c.to_stop_id, c.to_stop_name, s.geometry;
    """
    
    # Load the query result into a GeoDataFrame
    segment_gdf = gpd.read_postgis(query, conn, geom_col="geometry")
    
    return segment_gdf

    
# Connect to the PostgreSQL/PostGIS database
conn = psycopg2.connect(host=db_host, dbname=db_name, user=db_user, password=db_pass)

# Load segments directly from PostGIS
segment_gdf = load_segments_from_postgis(conn)

# Close the database connection
conn.close()

In [ ]:
#Script for Section 11.2.4 (Visualization)
import branca.colormap as cm

# Function to visualize segments on a map
def visualize_segments(segment_data, cutoff=10000):
    """
    Visualizes the segments on a Folium map, where the color intensity
    gets darker for segments with a higher trip count (with a cutoff), and a legend is shown.
    
    Parameters:
    - segment_data: GeoDataFrame containing segment information and geometries
    - cutoff: Maximum value for the trip count to control color intensity
    
    Returns:
    - Folium map object displayed in the notebook
    """
    # Initialize a Folium map centered on Genova
    transport_map = fl.Map(location=[44.4264, 8.9151], tiles='CartoDB positron', zoom_start=12)

    # Define the colormap from white (low count) to red (high count), with a cutoff at 6000
    colormap = cm.LinearColormap(colors=['white', 'red'], vmin=0, vmax=cutoff, caption='Trip Count')
    colormap.add_to(transport_map)

    # Loop over each segment and add it to the map
    for _, segment in segment_data.iterrows():
        # Apply cutoff: any count above the cutoff is set to the maximum (cutoff value)
        trip_count = min(segment['count'], cutoff)
        
        # Get color based on trip count
        color = colormap(trip_count)
        
        # Convert the geometry to GeoJSON with associated properties
        geo_json = fl.GeoJson(
            data={
                "type": "Feature",
                "geometry": segment['geometry'].__geo_interface__,
                "properties": {
                    "routes": segment['routes'],
                    "from_stop_name": segment['from_stop_name'],
                    "to_stop_name": segment['to_stop_name'],
                    "count": segment['count']
                }
            },
            style_function=lambda x, color=color: {
                'color': color,
                'weight': 3,
                'opacity': 0.7
            },
            tooltip=GeoJsonTooltip(
                fields=['routes', 'from_stop_name', 'to_stop_name', 'count'],
                aliases=['Routes', 'From Stop', 'To Stop', 'Trip Count'],
                localize=True
            )
        )
        geo_json.add_to(transport_map)
    
    return transport_map

# Visualize the segments on a map with color intensity, cut off, and tooltips
transport_map = visualize_segments(segment_gdf)

# Display the map in the notebook
display(transport_map)

In [ ]:
#Script for Section 11.2.5.

import psycopg2
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

# Fetch detailed segment speed data
def fetch_segment_speeds():
    """
    Fetches the detailed speed values per segment from the PostGIS database.
    
    Returns:
    - DataFrame containing speed values and segment details
    """
    conn = psycopg2.connect(host=db_host, port=db_port, dbname=db_name, user=db_user, password=db_pass)
    query = """
    SELECT (s.distance_m / EXTRACT(EPOCH FROM (c.t_arrival - c.t_departure)) * 3.6) AS speedKMH,
           c.from_stop_id, c.from_stop_name, c.to_stop_id, c.to_stop_name
    FROM connections AS c
    JOIN segments AS s
    ON c.route_id=s.route_id 
       AND c.direction_id=s.direction_id 
       AND c.from_stop_id=s.start_stop_id 
       AND c.to_stop_id=s.end_stop_id
    WHERE date BETWEEN '2024-09-01' AND '2024-09-30' 
      AND EXTRACT(EPOCH FROM (c.t_arrival - c.t_departure)) > 0;
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    # Add a column that concatenates 'from_stop_name' and 'to_stop_name' for easy selection
    df['segment'] = df['from_stop_name'] + " -> " + df['to_stop_name']
    
    return df

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Load data
speed_df = fetch_segment_speeds()

# Layout of the Dash app
app.layout = dbc.Container([
    html.H1("Speed Distribution per Segment"),
    
    # Dropdown for selecting segment
    dcc.Dropdown(
        id="segment-dropdown",
        options=[{"label": seg, "value": seg} for seg in sorted(speed_df['segment'].unique())],
        value=speed_df['segment'].unique()[0],  # Default value
        clearable=False,
        style={"width": "80%"}
    ),
    
    # Graph to display speed distribution
    dcc.Graph(id="speed-distribution-plot")
])

# Callback to update the graph based on selected segment
@app.callback(
    Output("speed-distribution-plot", "figure"),
    Input("segment-dropdown", "value")
)
def update_graph(selected_segment):
    # Filter data for the selected segment
    segment_df = speed_df[speed_df['segment'] == selected_segment]
    
    # Create histogram for the speed distribution
    fig = px.histogram(segment_df, x="speedkmh", nbins=30, title=f"Speed Distribution for {selected_segment}",
                       labels={"speedkmh": "Speed (km/h)"})
    
    # Ensure the bar doesn't stretch too wide by setting a reasonable x-axis range
    if not segment_df['speedkmh'].empty:
        speed_min = segment_df['speedkmh'].min() - 5  # Padding for better visibility
        speed_max = segment_df['speedkmh'].max() + 5
        fig.update_xaxes(range=[speed_min, speed_max])
        
    fig.update_layout(xaxis_title="Speed (km/h)", yaxis_title="Frequency")
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)